# Poisson editing

The purpose of this notebook is to apply the methods of image editing proposed by Pérez, Gangnet and Blake in their study *Poisson image editing* [1]. It is based on a notebook adapted by Nicolas Papadakis (IMB) and Charles Dossal (INSA Toulouse) from works by William Emmanuel and Pierre Bénard (LaBRI).

This work has been realized by Albin Cintas, Clément Targe, Baptiste Aussel and Quentin Thuet (ModIA 2020/2023 - ENSEEIHT/INSA)

## Introduction
The goal of this assignment is to apply the Poisson editing algorithm [1] for image blending.

In the following, $T$ is a target image,  $S$ a source image,  and a binary mask representing an area $\Omega$ of $S$ to copy in $T$. All images are defined on the same domain $D$ of size $M\times N$.

What is the optimization problem we are looking to solve ?   
  
$$\min_u \int_\Omega ||\nabla u-v||^2 = \min_u ||\nabla u-v||_{2}^2$$
under the constraint $u_{|\partial \Omega}=T$. $v$ is here a vector field that will depend on the fusion we want to do. 

We will first study Seemless Cloning. Then we will see other ways to fusion images with certain effects. 


<table align="center"><tr><td><img src="./sources_ondelettes/target_Boat.png" style="width: 200px;"></td><td><img src="./sources_ondelettes/source_Kraken.png" style="width: 200px;"></td><td><img src="./sources_ondelettes/mask_Kraken.png" style="width: 200px;"></td><td><img src="./img/naive.png" style="width: 200px;"></td><td><img src="./sources_ondelettes/poisson_blending.png" style="width: 200px;"></td></tr>
<tr><td>Target $T$</td><td>Source $S$</td><td>Mask</td><td>Naive copy/paste of $S$</td><td>Poisson blending [1]: seemless cloning</td></tr>
</table>


We will study two examples but you can try others on your own.
The data can be found at the following adress :

http://dl.free.fr/rm.pl?h=bOSoNRP4l&i=93426714&s=mwWbk2t533WxFMxD47ERDuR9MlkDPwc8

In [1]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import logging
logging.getLogger("param").setLevel(logging.CRITICAL)
import holoviews as hv
import param
import panel as pn
import matplotlib.pyplot as plt
import requests
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import sys
import time

ModuleNotFoundError: No module named 'pywt'

In [ ]:
import logging
logging.getLogger("param").setLevel(logging.CRITICAL)

In [ ]:
caselist=['Kraken', 'MonaLisa',"GradF"]

Data are available online but you have downloaded them, you can work online changing the value of the variable "local" in the next cell.

In [ ]:
def chargeData(name):
    if name=='Kraken':
        target=np.array(Image.open("sources_ondelettes/target_Boat.png")).astype(float)
        source=np.array(Image.open("sources_ondelettes/source_Kraken.png")).astype(float)
        mask2=np.array(Image.open("sources_ondelettes/mask_Kraken.png")).astype(float)/255
    if name=='MonaLisa':
        target=np.array(Image.open("sources_ondelettes/Joconde.jpg")).astype(float)
        source=np.array(Image.open("sources_ondelettes/source_Heisenberg.jpeg")).astype(float)
        mask2=np.array(Image.open("sources_ondelettes/mask_joconde.jpeg")).astype(float)/255
    if name=='GradF':
        target=np.array(Image.open("sources_ondelettes/0_grad_target.png")).astype(float)[2:,4:,:]
        source=np.array(Image.open("sources_ondelettes/0_grad_source.png")).astype(float)[3:,4:,:]
        mask2=np.array(Image.open("sources_ondelettes/0_grad_mask.png")).astype(float)/255
        mask2 = mask2[:,:,0]
    if name=='Ayoub_soleil':
        target=np.array(Image.open("sources_ondelettes/ayoub_target.png")).astype(float)[1:,:,:]
        source=np.array(Image.open("sources_ondelettes/ayoub_source2.png")).astype(float)[:674,:,:]
        mask2=np.array(Image.open("sources_ondelettes/ayoub_mask2.png")).astype(float)/255
        mask2 = mask2[:674,:,0]
    if name=='Ayoub_galet':
        target=np.array(Image.open("sources_ondelettes/ayoub_target2.png")).astype(float)[:,:,:]
        source=np.array(Image.open("sources_ondelettes/ayoub_source2.png")).astype(float)[:674,:,:]
        mask2=np.array(Image.open("sources_ondelettes/ayoub_mask2.png")).astype(float)/255
        mask2 = mask2[:674,:,0]
    if name=='Ayoub_texture':
        target=None
        source=np.array(Image.open("sources_ondelettes/ayoub_source3.png")).astype(float)[:,:,:]
        mask2=np.array(Image.open("sources_ondelettes/ayoub_mask3.png")).astype(float)/255
        mask2 = mask2[:,:,0]
    if name=='Orange':
        target=None
        source=np.array(Image.open("sources_ondelettes/orange_source.png")).astype(float)[:,:,:]
        mask2=np.array(Image.open("sources_ondelettes/orange_mask.png")).astype(float)/255
        mask2 = mask2[:,:,0]
    return target,source,mask2

Here are the input data : a target, a source and a mask.

In [ ]:
target,source,mask2=chargeData('Kraken')
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

In the following we are giving some discrete gradient and associated divergence. These gradients and divergences will be used to solve the optimization problem later. Indeed, these operators allow to compute the derivative of our functional.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y

In [ ]:
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

Below we define the projection of our source image on the destination, and the gradient of the function  f : $x\mapsto \Vert\nabla x -y\Vert_2^2$ :  
$$\nabla f(x) = 2 \times {\mbox{div}} (\nabla x - y)$$
$$$$
Reminder : if $F(x)=\frac{1}{2}\Vert A x -y\Vert_2^2$ with $A$ a linear operator, then $\nabla F(x) = A^{*}(A x - y)$ with $A^{*}$ the adjoint operator of $A$.  
Particularly, if $A:x\mapsto \nabla x$, then $A^{*}:x\mapsto {\mbox{div}} \ x$

In [ ]:
def Proj(source,mask,target):
    res = np.copy(target)
    res[np.where(mask==1)] = source[np.where(mask==1)]
    return res
def GradientFonc(x,y):
    res = 2.*Div(Gradient(np.array(x))-np.array(y))
    return res

We compute then a naive fusion with a simple projection.

In [ ]:
target,source,mask2=chargeData('Kraken')
optionsgray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(Proj(source,mask2,target).astype('uint8')).opts(**optionsRGB))

## Seamless cloning
We remind you that the optimization problem we want to solve is the following one :  
$$\min_u ||\nabla u-v||^2_2,$$
under the constraint $u_{|\partial \Omega}=T$

The idea of Seamless cloning is to copy the spatial gradients $\nabla S$ of the source image inside $T$, and not the color values $S$. 
To realize such blending, we find an image $u$ solution of:

$$\min_u ||\nabla u-\nabla S||^2_2,$$
under the constraint $u_{D\backslash \Omega}=T$. In fact we just take $v =\nabla S$.


We write a function FBPoissonEditing that solves the optimization problem presented above. The algorithm firstly used is the Forward-Backward algorithm. Lately we will complete it with an additional algorithm named FISTA that we will present later.  

In [ ]:
def PoissonEditing(targ,sour,ma,v,step=0.1,Niter=100,optimization_method="FB",alpha=3):
    x = Proj(sour,ma,targ)
    f = [np.linalg.norm(Gradient(np.array(x))-np.array(v))**2]
    for n in range(Niter):
        yn = np.copy(x)
        x = Proj(yn-step*GradientFonc(x,v),ma,targ)
        f += [np.linalg.norm(Gradient(np.array(x))-np.array(v))**2]
    return np.clip(x,0,255),f[10:]

Below we compute the PoissonEditing for the RGB images.

In [ ]:
def PoissonEditingColor(targ,sour,ma,step=0.1,Niter=100,editing_method="seamless",optimization_method="FB",alpha=3):
    v = Gradient(sour[:,:,0]),Gradient(sour[:,:,1]),Gradient(sour[:,:,2])
    out = np.zeros(targ.shape)
    out[:,:,0],f1 = PoissonEditing(targ[:,:,0],sour[:,:,0],ma,v[0],step,Niter,optimization_method,alpha)
    out[:,:,1],f2 = PoissonEditing(targ[:,:,1],sour[:,:,1],ma,v[1],step,Niter,optimization_method,alpha)
    out[:,:,2],f3 = PoissonEditing(targ[:,:,2],sour[:,:,2],ma,v[2],step,Niter,optimization_method,alpha)
    return out, (f1,f2,f3)

We now create a dashboard to perform the fusion in real time.

In [ ]:
class FBFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,1000))
    step = param.Number(0.1,bounds=(0.01,1))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoiss, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="seamless",optimization_method="FB")
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBPoiss=hv.RGB(imPoiss.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Forward-Backward",height=250,width=250)
        RGBNaive=hv.RGB(Proj(source,mask2,target).astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Projection naïve",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBPoiss,RGBNaive))

In [ ]:
fbfusion= FBFusion()
pn.Row(fbfusion.param,fbfusion.view)

In order to be more efficient, we compute a Fusion using FISTA with a parameter $\alpha$ and create the associated dashboard. 
FISTA is defined as a Forward-Backward algorithm combined with Nesterov acceleration. Indeed, classical Forward Backward consists in updating $x_n$ as follows : $$x_{n+1} = x_n - \gamma \nabla F(x_n)$$  
But in FISTA we change this iteration procedure with :  
$$
\left\{
    \begin{array}{ll}
        y_n = x_n + \frac{n}{n+\alpha}\times (x_n - x_{n-1}) \\
        x_{n+1} = y_n - \gamma \nabla F(x_n)
    \end{array}
\right.
$$

In [ ]:
def PoissonEditing(targ,sour,ma,v,step=0.1,Niter=100,optimization_method="FB",alpha=3):
    y = Gradient(sour)
    if optimization_method == "FISTA":
        x0 = Proj(sour,ma,targ)
    x1 = Proj(sour,ma,targ)
    f = [np.linalg.norm(Gradient(np.array(x1))-np.array(v))**2]
    for n in range(Niter):
        yn = np.copy(x1)
        if optimization_method == "FISTA":
            yn += (n / (n+alpha)) * (x1 - x0)
            x0 = np.copy(x1)
        x1 = Proj(yn-step*GradientFonc(x1,v),ma,targ)
        f += [np.linalg.norm(Gradient(np.array(x1))-np.array(v))**2]
    return np.clip(x1,0,255),f[10:]

In [ ]:
class FISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,1000))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoiss, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="seamless",optimization_method="FISTA",alpha=self.alpha)
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBFISTA=hv.RGB(imPoiss.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBNaive=hv.RGB(Proj(source,mask2,target).astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Projection naïve",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBFISTA,RGBNaive))

Let's look at the real time fusion using FISTA.

In [ ]:
fistafusion=FISTAFusion()
pn.Row(fistafusion.param,fistafusion.view)

Now, we compare the two methods for a fixed number of iterations.

In [ ]:
class FBvsFISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    canalplot = param.ObjectSelector(default='R',objects=['R','G','B'])
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoissFISTA, (f1FISTA,f2FISTA,f3FISTA) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="seamless",optimization_method="FISTA",alpha=self.alpha)
        curve1FISTA=hv.Curve(f1FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='b')
        curve2FISTA=hv.Curve(f2FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='g')
        curve3FISTA=hv.Curve(f3FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='r')
        imPoissFB, (f1FB,f2FB,f3FB) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="seamless",optimization_method="FB")
        curve1FB=hv.Curve(f1FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='b',line_dash="dashed")
        curve2FB=hv.Curve(f2FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='g',line_dash="dashed")
        curve3FB=hv.Curve(f3FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='r',line_dash="dashed")
        
        RGBPoissFISTA=hv.RGB(imPoissFISTA.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBPoissFB=hv.RGB(imPoissFB.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Forward-Backward",height=250,width=250)
        if self.canalplot == 'R':
            curves = curve1FISTA*curve1FB 
            curves.opts(title="Comparaison pour le canal rouge et " + str(self.Niter) + " itérations",height=250)
        elif self.canalplot == 'G':
            curves = curve2FISTA*curve2FB
            curves.opts(title="Comparaison pour le canal vert et " + str(self.Niter) + " itérations",height=250)
        elif self.canalplot == 'B':
            curves = curve3FISTA*curve3FB
            curves.opts(title="Comparaison pour le canal bleu et " + str(self.Niter) + " itérations",height=250)
        else:
            raise ValueError 
        return pn.Column(curves,pn.Row(RGBPoissFISTA,RGBPoissFB))

In [ ]:
fbvsfista=FBvsFISTAFusion()
pn.Row(fbvsfista.param,fbvsfista.view)

## Mixing gradient
We remind you again that the optimization problem we want to solve is the following one :  
$$\min_u ||\nabla u-v||^2_2,$$
under the constraint $u_{|\partial \Omega}=T$

This time, we want that some details from the target appear on the finale image such as background or transparent effects. This means that we should integrate target gradient when we define $v$, our vector field. 
So the idea is now to define $v$ as :   
$$
\forall x \in \Omega,  v(x) = \left\{
    \begin{array}{ll}
        \nabla T(x) & \mbox{if } |\nabla T(x)| > |\nabla S(x)| \\
        \nabla S(x) & \mbox{otherwise}
    \end{array}
\right.
$$

We define the new function $v$.

In [ ]:
def v_MG(x, y, ma):
    
    x_hor_abs = np.abs(x[0])
    x_vert_abs = np.abs(x[1])
    y_hor_abs = np.abs(y[0])
    y_vert_abs = np.abs(y[1])
    
    condition_hor = np.where(x_hor_abs > y_hor_abs)
    condition_vert = np.where(x_vert_abs > y_vert_abs)
    
    v = np.copy(y) 
    v[0][condition_hor] = x[0][condition_hor]
    v[1][condition_vert] = x[1][condition_vert]
    
    indice_mask = np.where(ma==0)
    v[0][indice_mask] = y[0][indice_mask]
    v[1][indice_mask] = y[1][indice_mask]
    
    return v

Now, let's adapt our Forward-Backward function to this new $v$.

In [ ]:
def PoissonEditingColor(targ,sour,ma,step=0.1,Niter=100,editing_method="seamless",optimization_method="FB",alpha=3):
    if editing_method == "seamless":
        v = Gradient(sour[:,:,0]),Gradient(sour[:,:,1]),Gradient(sour[:,:,2])
    elif editing_method == "mixing":
        v = v_MG(Gradient(targ[:,:,0]), Gradient(sour[:,:,0]),ma),\
            v_MG(Gradient(targ[:,:,1]), Gradient(sour[:,:,1]),ma),\
            v_MG(Gradient(targ[:,:,2]), Gradient(sour[:,:,2]),ma)
    out = np.zeros(targ.shape)
    out[:,:,0],f1 = PoissonEditing(targ[:,:,0],sour[:,:,0],ma,v[0],step,Niter,optimization_method,alpha)
    out[:,:,1],f2 = PoissonEditing(targ[:,:,1],sour[:,:,1],ma,v[1],step,Niter,optimization_method,alpha)
    out[:,:,2],f3 = PoissonEditing(targ[:,:,2],sour[:,:,2],ma,v[2],step,Niter,optimization_method,alpha)
    return out, (f1,f2,f3)

For the mixing gradient, the following example is the most appropriate.

In [ ]:
target,source,mask2=chargeData("GradF")

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB,title="Target"),\
       hv.RGB(source.astype('uint8')).opts(**optionsRGB,title="Source"),\
       hv.Image((mask2*255).astype('uint8')).opts(**optionsGray,title="Mask"))

Let's see the effect of the mixing gradient with Forward-Backward and with FISTA.

In [ ]:
class FBFusionMG(param.Parameterized):
    case = param.ObjectSelector(default='GradF',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(0.1,bounds=(0.01,1))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoiss, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="mixing",optimization_method="FB")
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBPoiss=hv.RGB(imPoiss.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Forward-Backward",height=250,width=250)
        RGBNaive=hv.RGB(Proj(source,mask2,target).astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Projection naïve",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBPoiss,RGBNaive))

In [ ]:
fbmg=FBFusionMG()
pn.Row(fbmg.param,fbmg.view)

In [ ]:
class FISTAMG(param.Parameterized):
    case = param.ObjectSelector(default='GradF',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoiss2, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="mixing",optimization_method="FISTA",alpha=self.alpha)
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBFISTA=hv.RGB(imPoiss2.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBNaive=hv.RGB(Proj(source,mask2,target).astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Projection naïve",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBFISTA,RGBNaive))

In [ ]:
fistamg=FISTAMG()
pn.Row(fistamg.param,fistamg.view)

We also create a dashboard to compare the performance of Forward-Backward and of FISTA

In [ ]:
class FBvsFISTAFusionMG(param.Parameterized):
    case = param.ObjectSelector(default='GradF',objects=caselist)
    canalplot = param.ObjectSelector(default='R',objects=['R','G','B'])
    Niter = param.Integer(100,bounds=(10,1000))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        imPoissFISTA, (f1FISTA,f2FISTA,f3FISTA) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="mixing",optimization_method="FISTA",alpha=self.alpha)
        curve1FISTA=hv.Curve(f1FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='b')
        curve2FISTA=hv.Curve(f2FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='g')
        curve3FISTA=hv.Curve(f3FISTA,kdims='x',vdims='v',label="FISTA").opts(width=500,color='r')
        imPoissFB = np.zeros(target.shape)
        imPoissFB, (f1FB,f2FB,f3FB) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="mixing",optimization_method="FB")
        curve1FB=hv.Curve(f1FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='b',line_dash="dashed")
        curve2FB=hv.Curve(f2FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='g',line_dash="dashed")
        curve3FB=hv.Curve(f3FB,kdims='x',vdims='v',label="Forward Backward").opts(width=500,color='r',line_dash="dashed")
        
        RGBPoissFISTA=hv.RGB(imPoissFISTA.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBPoissFB=hv.RGB(imPoissFB.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Forward-Backward",height=250,width=250)
        if self.canalplot == 'R':
            curves = curve1FISTA*curve1FB 
            curves.opts(title="Comparaison pour le canal rouge et " + str(self.Niter) + " itérations",height=250)
        elif self.canalplot == 'G':
            curves = curve2FISTA*curve2FB
            curves.opts(title="Comparaison pour le canal vert et " + str(self.Niter) + " itérations",height=250)
        elif self.canalplot == 'B':
            curves = curve3FISTA*curve3FB
            curves.opts(title="Comparaison pour le canal bleu et " + str(self.Niter) + " itérations",height=250)
        else:
            raise ValueError 
        return pn.Column(curves,pn.Row(RGBPoissFISTA,RGBPoissFB))

In [ ]:
fbvsfistamg=FBvsFISTAFusionMG()
pn.Row(fbvsfistamg.param,fbvsfistamg.view)

#### Limits of these methods

Here are two example showing the limits of the methods shown above. In fact, mixing gradient works well when the following conditions are met:
* The gradient of the target is greater than the gradient of the background of the source
* The gradient of the target is lower than the gradient of the foreground of the source

In this first example, the gradient of the target is too low (it is lower than the gradient of the background of the source).

In [ ]:
target,source,mask2 = chargeData("Ayoub_soleil")
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

In [ ]:
step = 0.05
Niter = 200
alpha = 4
imPoiss2, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,step,Niter,editing_method="mixing",optimization_method="FISTA",alpha=3)
pn.Row(hv.RGB(imPoiss2.astype('uint8')).opts(width=1000,height=674))

In the second example, the gradient of the target is too high (it is greater than the gradient of the foreground of the source).

In [ ]:
target,source,mask2 = chargeData("Ayoub_galet")

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

In [ ]:
step = 0.05
Niter = 200
alpha = 4
imPoiss2, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,step,Niter,editing_method="mixing",optimization_method="FISTA",alpha=3)
pn.Row(hv.RGB(imPoiss2.astype('uint8')).opts(width=1000,height=674))

## Texture flattening 

Thereafter, we will implement another Poisson Editing effect : the texture flattening. As always, the only thing that changes is $v$ in our optimization problem. (reminder : (P) $\min_u ||\nabla u-v||^2_2,$ under the constraint $u_{|\partial \Omega}=T$)  
$$$$
Here the v is defined as : 
$$
v(x) = M(x) \times \nabla T(x)
$$
where M is a binary matrix turned on at a few locations of interest. A matrix M that is interesting is an edge detector. 

We will define below this edge detector, and then we will implement this new method. 

In [ ]:
def edge_detector(image, seuil, mask):
    M = np.zeros(image.shape)
    r = image[:,:,0]
    g = image[:,:,1]
    b = image[:,:,2]
    r_roll = np.roll(r, 1, axis = 0)
    g_roll = np.roll(g, 1, axis = 0)
    b_roll = np.roll(b, 1, axis = 0)
    diff1 = np.abs(r-r_roll)
    diff2 = np.abs(g-g_roll)
    diff3 = np.abs(b-b_roll)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if diff1[i][j] + diff2[i][j] + diff3[i][j] > seuil :
                M[i][j] = 1
    M[:,:,0] = M[:,:,0] * (mask) + 1 - mask
    M[:,:,1] = M[:,:,1] * (mask) + 1 - mask
    M[:,:,2] = M[:,:,2] * (mask) + 1 - mask
    return np.clip(M, 0, 1) 

In [ ]:
_,source,mask_text = chargeData("Ayoub_texture")

In [ ]:
M = edge_detector(source, 10, mask_text)
plt.imshow(M)
plt.show()

Below you can find the implementation of the new $v$.

In [ ]:
def v_texture(image, mask):
    M = edge_detector(image, 10, mask)
    grad = Gradient(image)
    return [M * grad[0], M * grad[1]]

Now we can adapt our Poisson Editing method with the new effect. 

In [ ]:
def PoissonEditingColor(targ,sour,ma,step=0.1,Niter=100,editing_method="seamless",optimization_method="FB",alpha=3, mask_texture=None):
    if editing_method == "seamless":
        v = Gradient(sour[:,:,0]),Gradient(sour[:,:,1]),Gradient(sour[:,:,2])
    elif editing_method == "mixing":
        v = v_MG(Gradient(targ[:,:,0]), Gradient(sour[:,:,0]),ma),\
            v_MG(Gradient(targ[:,:,1]), Gradient(sour[:,:,1]),ma),\
            v_MG(Gradient(targ[:,:,2]), Gradient(sour[:,:,2]),ma)
    elif editing_method == "texture":
        targ = sour
        mask = np.ones(source.shape[:2])
        if mask_texture is None:
            mask_texture = np.ones(targ.shape[:2])
        v_hor, v_vert = v_texture(targ, mask_texture)
        v = (v_hor[:,:,0], v_vert[:,:,0]),\
            (v_hor[:,:,1], v_vert[:,:,1]),\
            (v_hor[:,:,2], v_vert[:,:,2])
    
        
    out = np.zeros(targ.shape)
    out[:,:,0],f1 = PoissonEditing(targ[:,:,0],sour[:,:,0],ma,v[0],step,Niter,optimization_method,alpha)
    out[:,:,1],f2 = PoissonEditing(targ[:,:,1],sour[:,:,1],ma,v[1],step,Niter,optimization_method,alpha)
    out[:,:,2],f3 = PoissonEditing(targ[:,:,2],sour[:,:,2],ma,v[2],step,Niter,optimization_method,alpha)
    return out, (f1,f2,f3)

As the previous results showed that FISTA goes undoubtly faster than simple Forward-Backward, we decide from now on to use only FISTA in order to solve our problems. 

In [ ]:
class FISTATexture(param.Parameterized):
    Niter = param.Integer(100,bounds=(10,500))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData("Ayoub_texture")
        imPoiss, (f1,f2,f3) = PoissonEditingColor(target,source,mask_text,self.step,self.Niter,editing_method="texture",optimization_method="FISTA",mask_texture=mask_text)
        RGBPoiss=hv.RGB(imPoiss.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Texture - FISTA ("+ str(self.Niter) + " iter)",height=480,width=250)
        RGBNaive=hv.RGB(source.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Image originale",height=480,width=250)
        return pn.Column(pn.Row(RGBPoiss,RGBNaive))

In [ ]:
fistatxt=FISTATexture()
pn.Row(fistatxt.param,fistatxt.view)

## Local illumination changes

To finish, we propose a last effect that consists in improving luminosity on over and under luminous locations. 
As always, the only thing that changes is $v$ in our optimization problem. (reminder : (P) $\min_u ||\nabla u-v||^2_2,$ under the constraint $u_{|\partial \Omega}=T$)  

Here the v is defined as : 
$$
v = \alpha ^{\beta} |\nabla T|^{-\beta} \nabla T 
$$
with alpha and beta that can be initialized as $\alpha = 0.2 \times \mbox{ average gradient norm of T}$ and $\beta = 0.2$, but under you will be able to change them and see the results. 

In [ ]:
_,source,mask2=chargeData("Orange")

In [ ]:
def v_illuminati(image,mask):
    
    beta = 0.2
    
    grad = Gradient(image)
    normgrad = np.sqrt(grad[0]**2+grad[1]**2)
    alpha = 0.2*np.sum(normgrad*mask)/np.sum(mask)
    
    coef = alpha**beta*np.linalg.norm(grad[0]+grad[1])**(-beta)
    
    return coef*grad[0],coef*grad[1]

As usual, we adapt our function so that it takes the new method into account.

In [ ]:
def PoissonEditingColor(targ,sour,ma,step=0.1,Niter=100,editing_method="seamless",optimization_method="FB",alpha=3, mask_texture=None):
    if editing_method == "seamless":
        v = Gradient(sour[:,:,0]),Gradient(sour[:,:,1]),Gradient(sour[:,:,2])
    elif editing_method == "mixing":
        v = v_MG(Gradient(targ[:,:,0]), Gradient(sour[:,:,0]),ma),\
            v_MG(Gradient(targ[:,:,1]), Gradient(sour[:,:,1]),ma),\
            v_MG(Gradient(targ[:,:,2]), Gradient(sour[:,:,2]),ma)
    elif editing_method == "texture":
        targ = sour
        if mask_texture is None:
            mask_texture = np.ones(targ.shape[:2])
        v_hor, v_vert = v_texture(targ, mask_texture)
        v = (v_hor[:,:,0], v_vert[:,:,0]),\
            (v_hor[:,:,1], v_vert[:,:,1]),\
            (v_hor[:,:,2], v_vert[:,:,2])
    elif editing_method == "illumination":
        targ = sour
        v = (v_illuminati(targ[:,:,0],ma)),\
            (v_illuminati(targ[:,:,1],ma)),\
            (v_illuminati(targ[:,:,2],ma))
        
    out = np.zeros(targ.shape)
    out[:,:,0],f1 = PoissonEditing(targ[:,:,0],sour[:,:,0],ma,v[0],step,Niter,optimization_method,alpha)
    out[:,:,1],f2 = PoissonEditing(targ[:,:,1],sour[:,:,1],ma,v[1],step,Niter,optimization_method,alpha)
    out[:,:,2],f3 = PoissonEditing(targ[:,:,2],sour[:,:,2],ma,v[2],step,Niter,optimization_method,alpha)
    return out, (f1,f2,f3)

Let's visualize the effect of this new method.

In [ ]:
class FISTAIllum(param.Parameterized):
    Niter = param.Integer(30,bounds=(10,100))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData("Orange")
        imPoiss2, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method="illumination",optimization_method="FISTA",alpha=self.alpha)
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBFISTA=hv.RGB(imPoiss2.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBNaive=hv.RGB(source.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Image originale",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBFISTA,RGBNaive))

In [ ]:
fistaill=FISTAIllum()
pn.Row(fistaill.param,fistaill.view)

## Summary

Here are the final functions and a dashboard summarizing all methods.

In [ ]:
def PoissonEditing(targ,sour,ma,v,step=0.1,Niter=100,optimization_method="FB",alpha=3):
    y = Gradient(sour)
    if optimization_method == "FISTA":
        x0 = Proj(sour,ma,targ)
    x1 = Proj(sour,ma,targ)
    f = [np.linalg.norm(Gradient(np.array(x1))-np.array(v))**2]
    for n in range(Niter):
        yn = np.copy(x1)
        if optimization_method == "FISTA":
            yn += (n / (n+alpha)) * (x1 - x0)
            x0 = np.copy(x1)
        x1 = Proj(yn-step*GradientFonc(x1,v),ma,targ)
        f += [np.linalg.norm(Gradient(np.array(x1))-np.array(v))**2]
    return np.clip(x1,0,255),f[10:]

In [ ]:
def PoissonEditingColor(targ,sour,ma,step=0.1,Niter=100,editing_method="seamless",optimization_method="FB",alpha=3, mask_texture=None):
    if editing_method == "seamless":
        v = Gradient(sour[:,:,0]),Gradient(sour[:,:,1]),Gradient(sour[:,:,2])
    elif editing_method == "mixing":
        v = v_MG(Gradient(targ[:,:,0]), Gradient(sour[:,:,0]),ma),\
            v_MG(Gradient(targ[:,:,1]), Gradient(sour[:,:,1]),ma),\
            v_MG(Gradient(targ[:,:,2]), Gradient(sour[:,:,2]),ma)
    elif editing_method == "texture":
        targ = sour
        if mask_texture is None:
            mask_texture = np.ones(targ.shape[:2])
        v_hor, v_vert = v_texture(targ, mask_texture)
        v = (v_hor[:,:,0], v_vert[:,:,0]),\
            (v_hor[:,:,1], v_vert[:,:,1]),\
            (v_hor[:,:,2], v_vert[:,:,2])
    elif editing_method == "illumination":
        targ = sour
        v = (v_illuminati(targ[:,:,0],ma)),\
            (v_illuminati(targ[:,:,1],ma)),\
            (v_illuminati(targ[:,:,2],ma))
        
    out = np.zeros(targ.shape)
    out[:,:,0],f1 = PoissonEditing(targ[:,:,0],sour[:,:,0],ma,v[0],step,Niter,optimization_method,alpha)
    out[:,:,1],f2 = PoissonEditing(targ[:,:,1],sour[:,:,1],ma,v[1],step,Niter,optimization_method,alpha)
    out[:,:,2],f3 = PoissonEditing(targ[:,:,2],sour[:,:,2],ma,v[2],step,Niter,optimization_method,alpha)
    return out, (f1,f2,f3)

In [ ]:
class PoissonEditingView(param.Parameterized):
    Editing_Method = param.ObjectSelector(default='seamless',objects=['seamless','mixing','texture','illumination'])
    Optimization_Method = param.ObjectSelector(default='FISTA',objects=['FB','FISTA'])
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(0.1,bounds=(0.01,1))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        if self.Editing_Method == "seamless":
            target,source,mask2=chargeData("Kraken")
        if self.Editing_Method == "mixing":
            target,source,mask2=chargeData("GradF")
        if self.Editing_Method == "texture":
            target,source,mask2=chargeData("Ayoub_texture")
        if self.Editing_Method == "illumination":
            target,source,mask2=chargeData("Orange")
        imPoiss2, (f1,f2,f3) = PoissonEditingColor(target,source,mask2,self.step,self.Niter,editing_method=self.Editing_Method,optimization_method=self.Optimization_Method,alpha=self.alpha,mask_texture=mask2)
        curve1=hv.Curve(f1,kdims='x',vdims='v').opts(width=500,color='b')
        curve2=hv.Curve(f2,kdims='x',vdims='v').opts(width=500,color='g')
        curve3=hv.Curve(f3,kdims='x',vdims='v').opts(width=500,color='r')
        curves=curve1*curve2*curve3
        curves.opts(title="3 canaux de couleurs et " + str(self.Niter) + " itérations",height=250)
        RGBFISTA=hv.RGB(imPoiss2.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="FISTA",height=250,width=250)
        RGBNaive=hv.RGB(source.astype('uint8')).opts(xaxis=None,yaxis=None,toolbar=None,title="Image originale",height=250,width=250)
        return pn.Column(curves,pn.Row(RGBFISTA,RGBNaive))

In [ ]:
peview=PoissonEditingView()
pn.Row(peview.param,peview.view)